# Data exploration and cleaning

This notebook aims to clean data according to the desired population (see below).

In [ ]:
#Libs
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
import os

In [ ]:
DATA_RAW_DIR = os.path.join(os.getcwd(), os.pardir, 'data', 'raw')
print(DATA_RAW_DIR)

/work/churn_predictive_model/notebooks/../data/raw


In [ ]:
clients = pd.read_csv(DATA_RAW_DIR+'/clients_table.txt')

In [ ]:
clients.head()

,CustomerId,Surname,Geography,Gender,HasCrCard,IsActiveMember,EstimatedSalary,application_date,exit_date,birth_date
0,15745584,EIRLS,Germany,Female,0.0,1.0,0.00,2018-12-14,NaN,1997-09-18
1,14990118,MOLOCK,Italy,Male,1.0,0.0,121219.28,2019-01-08,NaN,1980-08-03
2,14733224,PAWLUCH,Italy,Female,1.0,1.0,159663.59,2012-08-01,2013-08-09,1977-08-19
3,14648573,NALLS,Spain,Male,1.0,0.0,140827.98,2019-06-19,NaN,1979-02-27
4,15365443,EBERLE,Italy,Male,1.0,0.0,35521.28,2014-01-26,2015-12-04,1972-12-21


In [ ]:
clients.dtypes

CustomerId            int64
Surname              object
Geography            object
Gender               object
HasCrCard           float64
IsActiveMember      float64
EstimatedSalary     float64
application_date     object
exit_date            object
birth_date           object
dtype: object

## Desired population 

* Contracts from 2015 onwards.
* Operations in Italy were closed in 2019.
* Take out clients with more than 75% of their info missing.
* Each client should only have one contract in the database, no more.
* Remember that for a client to be eligible, it must have at least two years of information within the company (even if it canceled the product).


In [ ]:
clients['application_date'] = pd.to_datetime(clients['application_date'])
clients['exit_date'] = pd.to_datetime(clients['exit_date'])
clients['birth_date'] = pd.to_datetime(clients['birth_date'])

In [ ]:
clients['application_date_year'] = clients['application_date'].dt.year
clients['application_date_month'] = clients['application_date'].dt.month
clients['application_date_day'] = clients['application_date'].dt.day

In [ ]:
clients['exit_date_year'] = clients['exit_date'].dt.year
clients['exit_date_month'] = clients['exit_date'].dt.month
clients['exit_date_day'] = clients['exit_date'].dt.day

In [ ]:
clients['birth_date_year'] = clients['birth_date'].dt.year
clients['birth_date_month'] = clients['birth_date'].dt.month
clients['birth_date_day'] = clients['birth_date'].dt.month

In [ ]:
#Contracts from 2015 onwards
clients = clients.loc[clients['application_date_year'] >= 2015]

In [ ]:
clients.shape

(623242, 16)

Dropping records before 2015, we got 623242 records.

Now lets delete people located in Italy, since the operations stopped in that country.

In [ ]:
#Operations in Italy were closed in 2019
clients = clients.loc[clients['Geography'] != 'Italy']

In [ ]:
clients.shape

(487424, 16)

Once Italy is droped from dataset, we have 487424 records.

In [ ]:
clients.isnull().sum()

CustomerId                     0
Surname                    23362
Geography                  23349
Gender                     23349
HasCrCard                  23349
IsActiveMember             23349
EstimatedSalary            23349
application_date               0
exit_date                 382626
birth_date                 23349
application_date_year          0
application_date_month         0
application_date_day           0
exit_date_year            382626
exit_date_month           382626
exit_date_day             382626
dtype: int64

We'll use `notna()` as we see more than 75% of missing information and this can lead to bias.

In [ ]:
clients = clients[clients['EstimatedSalary'].notna()]

In [ ]:
clients.shape

(464075, 16)

Taking clients with more that 75% of info missing out gives us 464075 records.

In [ ]:
products = pd.read_csv(DATA_RAW_DIR+'/products_table.txt')

In [ ]:
products.head()

,ContractId,CustomerId,Products
0,5Xo9sCduOGo5EhBYhzKrSlLIA,14993808,Product B
1,3nQbqbnpTeSkYIJWT1UDe1GZ1,15560417,Product B
2,PD9KwDPOwyJWO6LJKPGBbkrCz,15406428,Product C
3,IhS7pz8lu07LCZSqYNlL0fuLP,15604313,Product B
4,ftUey4oVmkZj8RaBVFH6SNzVM,14952409,Product C


In [ ]:
products

,ContractId,CustomerId,Products
0,5Xo9sCduOGo5EhBYhzKrSlLIA,14993808,Product B
1,3nQbqbnpTeSkYIJWT1UDe1GZ1,15560417,Product B
2,PD9KwDPOwyJWO6LJKPGBbkrCz,15406428,Product C
3,IhS7pz8lu07LCZSqYNlL0fuLP,15604313,Product B
4,ftUey4oVmkZj8RaBVFH6SNzVM,14952409,Product C
...,...,...,...
3739187,gG68mjjODmTNM21QPEeAKr0DZ,14723530,Product C
3739188,yWf0PHzqSjJvadbdMji4XtX5T,15837225,Product D
3739189,5kerYoMKP03ebWSrloZBjmWKz,14661828,Product C
3739190,CdnDnTfN3YYOozLACqf9fBak1,15559545,Product D


In [ ]:
products.drop_duplicates(keep=False, subset=['CustomerId'], inplace=True)

In [ ]:
clients_and_products = pd.merge(clients, products, on='CustomerId', how='inner')

In [ ]:
clients_and_products.shape

(118897, 18)

In [ ]:
clients_and_products.isnull().sum()

CustomerId                    0
Surname                       2
Geography                     0
Gender                        0
HasCrCard                     0
IsActiveMember                0
EstimatedSalary               0
application_date              0
exit_date                 91750
birth_date                    0
application_date_year         0
application_date_month        0
application_date_day          0
exit_date_year            91750
exit_date_month           91750
exit_date_day             91750
ContractId                    0
Products                      0
dtype: int64

When we consider clients who have one contract only, there're 118897 records.

In [ ]:
credit_score = pd.read_csv(DATA_RAW_DIR+'/credit_score_table.txt')

In [ ]:
credit_score.head()

,CustomerId,Date,Score
0,14759722,2018-07,717
1,15907276,2011-09,562
2,15346641,2013-01,604
3,15595126,2018-08,469
4,15227788,2013-09,655


In [ ]:
credit_score.shape

(19500000, 3)

In [ ]:
credit_score['application_date_year'] = [int(year[:4]) for year in credit_score['Date']]

In [ ]:
credit_score['application_date_month'] = [int(month[-2:]) for month in credit_score['Date']]

In [ ]:
credit_score[credit_score['CustomerId'] == 15907276 ]

,CustomerId,Date,Score,application_date_year,application_date_month
1,15907276,2011-09,562,2011,9
351557,15907276,2012-01,593,2012,1
762892,15907276,2011-11,675,2011,11
877831,15907276,2012-06,617,2012,6
1525837,15907276,2012-02,666,2012,2
1806594,15907276,2012-04,768,2012,4
1866637,15907276,2011-08,806,2011,8
3540405,15907276,2011-12,624,2011,12
5371491,15907276,2012-08,795,2012,8
11381537,15907276,2011-10,613,2011,10


In [ ]:
clients_products_score = pd.merge(clients_and_products, credit_score, 
        on=['CustomerId', 'application_date_year', 'application_date_month'], 
        how='inner')

In [ ]:
transactions = pd.read_csv(DATA_RAW_DIR+'/transactions_table.txt')

In [ ]:
transactions.head()

,CustomerId,Transaction,Value
0,15653320,trans_4,-294.03
1,14955663,trans_3,1297.97
2,15218505,trans_4,191.28
3,15391934,trans_3,2299.07
4,14847291,trans_3,1612.10


In [ ]:
transactions_grouped = transactions.groupby(['CustomerId']).sum().reset_index()

In [ ]:
clients_products_score_transactions = pd.merge(clients_products_score, transactions_grouped, 
        on='CustomerId', 
        how='inner')

In [ ]:
elegible = []

days_elegible = (clients_products_score_transactions['exit_date'] - clients_products_score_transactions['application_date']).dt.days

for i in range(len(days_elegible)):
    if days_elegible[i] >= 2*365:
        elegible.append(1)
    elif days_elegible[i] == np.NaN:
        elegible.append(1)
    else:
        elegible.append(0)

In [ ]:
clients_products_score_transactions['eligible'] = elegible

In [ ]:
import math

In [ ]:
client_age = (clients_products_score_transactions['application_date'] - clients_products_score_transactions['birth_date']).dt.days / 365

In [ ]:
client_age = [math.floor(age) for age in client_age]

In [ ]:
clients_products_score_transactions['client_age'] = client_age

In [ ]:
clients_products_score_transactions.head()

,CustomerId,Surname,Geography,Gender,HasCrCard,IsActiveMember,EstimatedSalary,application_date,exit_date,birth_date,...,birth_date_year,birth_date_month,birth_date_day,ContractId,Products,Date,Score,Value,eligible,client_age
0,14648573,NALLS,Spain,Male,1.0,0.0,140827.98,2019-06-19,NaT,1979-02-27,...,1979.0,2.0,2.0,WWlyRDX8AsGnWUPYiYGjidGKI,Product B,2019-06,683,0.00,0,40
1,15165393,LABIANCA,Spain,Male,1.0,1.0,2612.65,2018-02-22,2019-06-11,1974-07-11,...,1974.0,7.0,7.0,orIBQM9pK7Z7AYBQok1jypK6t,Product D,2018-02,487,119657.24,0,43
2,15165393,LABIANCA,Spain,Male,1.0,1.0,2612.65,2018-02-22,2019-06-11,1974-07-11,...,1974.0,7.0,7.0,orIBQM9pK7Z7AYBQok1jypK6t,Product D,2018-02,487,119657.24,0,43
3,15982728,GOUDEAU,France,Male,0.0,1.0,66465.09,2018-02-02,2019-06-01,1972-12-18,...,1972.0,12.0,12.0,sdnOZ3fBpu1sKsYrZJE9tfcRP,Product B,2018-02,595,43353.50,0,45
4,15434700,STIMMELL,Germany,Male,1.0,0.0,138615.32,2018-06-06,NaT,1994-07-22,...,1994.0,7.0,7.0,dF2Z2DZoUYMmf0ST8keQjvKQL,Product A,2018-06,706,92357.90,0,23


## Export data

We have cleaned data according to the instructions given by desired population. Now, it's time to export that cleaned data to a file which can be analized and processed.

In [ ]:
DATA_PROCESSED_DIR = os.path.join(os.getcwd(), os.pardir, 'data', 'processed')
print(DATA_PROCESSED_DIR)

/work/churn_predictive_model/notebooks/../data/processed


In [ ]:
clients_products_score_transactions.to_csv(DATA_PROCESSED_DIR+'/cleaned_dataset.csv')

## Important
Raw data is too large to be uploaded on GitHub. However, processed data is small enough. That's why I'm creating an auxiliar directory in which the processed data will be stored. In other words, this directory will contain the same files as `data/processed`

In [ ]:
DATA_AUX_DIR = os.path.join(os.getcwd(), os.pardir, 'data_sent_github')
print(DATA_AUX_DIR)

/work/churn_predictive_model/notebooks/../data_sent_github


In [ ]:
clients_products_score_transactions.to_csv(DATA_AUX_DIR+'/cleaned_dataset.csv')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=fb2ec55e-ada8-4de4-93de-2d05f236c13b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>